-sandbox
<img width="280" src="https://files.training.databricks.com/images/Apache-Spark-Logo_TM_200px.png" style="float: left: margin: 20px"/>                   

<br>
> # Bronze para Silver

<br>
## 08 - Métricas diárias de estoque

- Calcular e escrever uma tabela que tenha os preços diários de **`alta, baixa e média`** por ação. 
- Crucial para os relatórios e para uma ferramenta visual que os investidores estão usando regularmente.

-sandbox
<h2 style="color:red">Instruções: </h2>

- o que estamos construindo agora: calculando estatísticas descritivas básicas para preços de ações
- por que é útil: dados agregados quase prontos para os negócios

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Configurações úteis<br>

In [0]:
%run ./Includes/14-ClassroomSetup

Preparing the Python environment...

Defining courseware-specific utility methods...

Defining user-facing utility methods...

Defining custom variables for this lesson...

Initializing Databricks Academy's testing framework...

...

Cleaning up the learning environment...no actions taken.

WARNING: This notebook was tested on DBR 7.0, but we found DBR 7.3. 
 Using an untested DBR may yield unexpected results and/or various errors 
 Please update your cluster configuration and/or download a newer version of this course before proceeding.

The following variables were defined for you: 
 financeDataPath : dbfs:/mnt/training/finance-org 
 The location of the finance dataset used in this capstone project. 
 
 sourceDataPath : dbfs:/mnt/training/finance-org/solutions 
 The location of the specific datasets used in this capstone project as source data. 
 
 silverStockOrdersPath : dbfs:/mnt/training/finance-org/solutions/silver/stock_orders 
 dbfs:/mnt/training/finance-org/solutions/silver/stock_orders 
 
 targetDirectory : dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/08_metricas_diarias_estoque_psp/silver/stock_order_daily_metrics 
 dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/08_metricas_diarias_estoque_psp/silver/stock_order_daily_metrics 
 The following database were created for you: 
 Now using the database identified by databaseName : 
 thomaz_thomazrossito_com_br_capstone_finance_08_metricas_diarias_estoque_psp 
 This is a private, per-notebook, database used to provide isolation from other users and exercises. 
 All done!

Declared the following function:
 realityCheck

All done!

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Etapas a serem concluídas:

<br>
- Ler a tabela de pedidos de estoque a partir do diretório **`silverStockOrdersPath`** para um DataFrame
- Selecionar os campos **`order_timestamp`**, **`ticker`**, **`price`**
- Crie uma nova coluna chamada **`data`** contendo apenas as informações `YYYY-MM-DD`, convertendo para o campo **`data_de_pedida`**
- Agregue o DataFrame por **`date`** e **`ticker`** para obter 3 métricas:
   * valor mínimo de **`preço`** denominado **`baixo`**, arredondado para a segunda casa decimal
   * valor máximo de **`preço`** denominado **`alto`**,  arredondado para a segunda casa decimal
   * valor médio de  **`preço`** denominado **`média`**, arredondado para a segunda casa decimal
- Ordene o resultado por **`data`** e **`ticker`**
- Grave o resultado no diretório **`targetDirectory`** como uma tabela Delta usando o modo de substituição
- Retorne o DataFrame resultante

O esquema do DataFrame deve ser:

|name|type|
|---|---|
|date|DateType|
|ticker|StringType|
|low|DoubleType|
|high|DoubleType|
|average|DoubleType|

In [0]:
def func_metricasDiaEstoque(spark, silverStockOrdersPath, targetDirectory):
  
  
  from pyspark.sql.window import Window
  from pyspark.sql.functions import min, max, avg, col, round
  
  
  # Ler a tabela de pedidos de estoque a partir do diretório "silverStockOrdersPath" para um DataFrame
  # Selecionar os campos "order_timestamp", "ticker", "price"
  # Crie uma nova coluna chamada "data" contendo apenas as informações "YYYY-MM-DD", convertendo para o campo "data_de_pedida"
  # Agregue o DataFrame por "date" e "ticker" para obter 3 métricashe second decimal place
  # Ordene o resultado por data e ticker
  df = (spark.read
             .format("delta")
             .load(silverStockOrdersPath)
             .select("order_timestamp", 
                     "ticker", 
                     "price")
             .withColumn("date", col("order_timestamp").cast("date")) 
             .groupBy("date", 
                      "ticker")
             .agg(round(min(col("price")), 2).alias("low"),
                  round(max(col("price")), 2).alias("high"),  
                  round(avg(col("price")), 2).alias("average"))
             .orderBy("date",
                      "ticker"))


  # Grave o DataFrame resultante NO DIRETÓRIO "targetDirectory" como uma tabela Delta usando o modo de substituição e a opção de substituição de esquema.
  df.write \
    .mode("overwrite") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .partitionBy("date") \
    .save(targetDirectory)

  
  # Retorna o Dataframe
  return df

In [0]:
finalDF = func_metricasDiaEstoque(spark, silverStockOrdersPath, targetDirectory)

In [0]:
display(finalDF)

date,ticker,low,high,average
2019-09-01,ADSK,110.8,112.18,111.52
2019-09-01,AEE,57.31,57.93,57.6
2019-09-01,AIG,60.32,61.23,60.74
2019-09-01,ALL,100.52,101.69,101.09
2019-09-01,AMP,172.97,175.88,174.53
2019-09-01,AMZN,1246.23,1257.97,1250.94
2019-09-01,APA,44.32,45.66,45.04
2019-09-01,APTV,92.29,93.57,93.0
2019-09-01,ATVI,65.02,66.75,65.83
2019-09-01,AXP,100.3,100.77,100.54


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Validação</h2>

In [0]:
realityCheck(func_metricasDiaEstoque, spark, silverStockOrdersPath, targetDirectory)

Points,Test,Result
1,Returns correct schema,
1,Returns DataFrame with correct number of rows,
1,Returns DataFrame with correct results,
1,Silver table in place,
1,Silver table has correct content,


d
# <img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Fim do pipeline de ingestão na camada ***`Silver`***

-sandbox

<a href="http://www.apache.org/">Apache Software Foundation</a> <p>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a>